# **前日の20:00までのデータを用いて翌日の00:00~23:30までの日射時間(sl)をMLPで予測する**

### **必要な関数・ライブラリ**

In [1]:
# データ加工・処理・分析モジュール
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import random

%matplotlib inline

import time

In [2]:
def set_time(dataframe, col_name):
    '''
    to_datetimeを使うための前処理
    '''
    dataframe[col_name] = dataframe[col_name].map(lambda x : transform_time(x))
    return dataframe

In [3]:
def transform_time(x):
    '''
    set_time内で使う関数
    to_datetimeで24時をサポートしないので00に変更する処理
    '''
    str_x = str(x)
    res = ''
    if str(x)[8:10] == '24':
        res = str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:8] + ' 00:'+str_x[10:12] 
    else:
        res = str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:8] + ' '+ str_x[8:10] +':'+str_x[10:12]
    return res

In [4]:
def normalize_array(x):
    '''
    min, max, min-max正規化を行なった配列(np.array)を返す
    '''
    x = np.array(x)
    x_min = x.min()
    x_max = x.max()
    normalized = (x - x_min) / (x_max - x_min) 
    return x_min, x_max, normalized

In [5]:
def denormalize_array(normalized_x, x_min, x_max):
    '''
    正規化前のmin, maxを用いて元のスケールに戻す
    '''
    normalized_x = np.array(normalized_x)
    denormalize_array = (normalized_x) * (x_max - x_min) + x_min
    return denormalize_array

In [40]:
def get_input_data(X):
    '''
    全時系列データから00:00 ~ 20:00までのデータごとに分割する
    '''
    input_list = []
    # 一日は48個の時間帯に分けられる
    total_size = len(X) // 48
    reshaped_size = X[0:41].shape[0] * X[0:41].shape[1]
    for i in range(total_size+1):
        each_data = X[(i*48):(i*48+41)]
        reshaped = each_data.reshape(reshaped_size)
        input_list.append(list(reshaped))
    input_list = np.array(input_list)
    return input_list

In [39]:
def get_output_data(Y):
    '''
    全時系列データから00:00 ~ 23:30までのデータごとに分割する
    '''
    output_list = []
    # 一日は48個の時間帯に分けられる
    total_size = len(Y) // 48
    for i in range(total_size):
        each_data = Y[(i*48):(i*48+48)]
        reshaped = each_data.reshape(48)
        output_list.append(list(reshaped))
    output_list = np.array(output_list)
    return output_list

In [8]:
def drop_nan(X, Y):
    '''
    正解データがnanであるデータの組を削除
    '''
    mask = np.isnan(Y)
    X = X[~mask]
    Y = Y[~mask]
    return X, Y

In [9]:
def calc_mae(X, Y):
    '''
    X, Yがメモリを食いすぎるのでfor文で計算
    '''
    mse = 0
    for i in range(len(X)):
        mse += np.abs(X[i]- Y[i])
    return mse/len(X)

### **データの準備**

In [57]:
# 予測する発電所番号を決める
target_place = 1
model_name = "mlp_"+str(target_place)
try:
    os.mkdir('./data/'+model_name)
except:
    print("file exists")

In [21]:
# アメダスデータの読み込み
if target_place == 1 or target_place == 2:
    # 名古屋アメダスのデータを使って予測する, amd_51106
    # 対象アメダスは横浜アメダス, amd_46106
    # 各amdidはamd_masterに記載されている
    amd_data = pd.read_csv('data/raw_data/amd_51106.tsv', delimiter = '\t')
    amd_data = set_time(amd_data, 'datetime')
    amd_data['datetime'] = amd_data['datetime'].map(lambda x : pd.to_datetime(x))
    target_amd_data = pd.read_csv('data/raw_data/amd_46106.tsv', delimiter = '\t')
    target_amd_data = set_time(target_amd_data, 'datetime')
    target_amd_data['datetime'] = target_amd_data['datetime'].map(lambda x : pd.to_datetime(x))
elif target_place == 3:
    # 甲府アメダスのデータを使って予測する, amd_49142
    # 各amdidはamd_masterに記載されている
    amd_data = pd.read_csv('data/raw_data/amd_49142.tsv', delimiter = '\t')
    amd_data = set_time(amd_data, 'datetime')
    amd_data['datetime'] = amd_data['datetime'].map(lambda x : pd.to_datetime(x))
else:
    raise ValueError("invalid input target_place_num")

In [24]:
# 30分毎のデータに編集
amd_data = amd_data[["datetime", "pr", "max_ws", "ave_ws", "max_tp", "min_tp", "sl"]]

# 欠損値を一つ前の値で置換/output_data
amd_data = amd_data.fillna(method='bfill')
amd_30 = amd_data.set_index('datetime').groupby(pd.TimeGrouper(freq='1800s', closed='left')).sum()

# datetimeのカラムを復活させる
amd_30['datetime'] = amd_30.index
amd_30.index = np.arange(len(amd_30))
amd_30.head()

,pr,max_ws,ave_ws,max_tp,min_tp,sl,datetime
0,0,68.0,41.0,73.0,68.0,0.0,2012-01-01 00:00:00
1,0,81.0,56.0,46.0,39.0,0.0,2012-01-01 00:30:00
2,0,81.0,52.0,43.0,40.0,0.0,2012-01-01 01:00:00
3,0,67.0,40.0,44.0,39.0,0.0,2012-01-01 01:30:00
4,0,33.0,20.0,43.0,37.0,0.0,2012-01-01 02:00:00


In [27]:
# 30分毎のデータに編集
target_amd_data = target_amd_data[["datetime", "sl"]]

# 欠損値を一つ前の値で置換/output_data
target_amd_data = target_amd_data.fillna(method='bfill')
target_30 = target_amd_data.set_index('datetime').groupby(pd.TimeGrouper(freq='1800s', closed='left')).sum()

# datetimeのカラムを復活させる
target_30['datetime'] = target_30.index
target_30.index = np.arange(len(target_30))
target_30.head()

,sl,datetime
0,0.0,2012-01-01 00:00:00
1,0.0,2012-01-01 00:30:00
2,0.0,2012-01-01 01:00:00
3,0.0,2012-01-01 01:30:00
4,0.0,2012-01-01 02:00:00


In [28]:
# モデル構築のためにデータを分割する

# 学習に必要なデータ
# 前日の00:00 ~ 20:00のデータで翌日の0:00 ~ 23:30を予測する
train_x_s_idx = amd_30[amd_30['datetime'] == pd.to_datetime('2012-01-01 00:00')].index[0]
train_x_e_idx = amd_30[amd_30['datetime'] == pd.to_datetime('2015-12-30 20:00')].index[0]
train_y_s_idx = amd_30[amd_30['datetime'] == pd.to_datetime('2012-01-02 00:00')].index[0]
train_y_e_idx = amd_30[amd_30['datetime'] == pd.to_datetime('2015-12-31 23:30')].index[0]

output_tp = target_30["sl"][train_y_s_idx:train_y_e_idx+1]
input_data = amd_30[[ "pr", "max_ws", "ave_ws", "max_tp", "min_tp", "sl"]][train_x_s_idx:train_x_e_idx+1]

# 予測に必要なデータ
# 前日の00:00 ~ 20:00のデータで翌日の0:00 ~ 23:30を予測する
# test_y_s_idx = amd_data[amd_data['datetime'] == pd.to_datetime('2015-12-31 00:00')].index[0]
# test_y_e_idx = amd_data[amd_data['datetime'] == pd.to_datetime('2017-3-31 23:50')].index[0]
# test_x_s_idx = test_y_s_idx - chunk_size
# test_x_e_idx = amd_data[amd_data['datetime'] == pd.to_datetime('2017-3-31 23:40')].index[0]

# test_input_tp = amd_data[test_x_s_idx:test_x_e_idx+1]
# test_output_tp = amd_data[test_y_s_idx:test_y_e_idx+1]

In [84]:
# MLPに突っ込むための準備をする

# rnnに突っ込むためにmin-max正規化しておく
normalized_input = (input_data - input_data.min()) / (input_data.max() - input_data.min())
normalized_input = np.array(normalized_input)
normalized_output = (output_tp - output_tp.min()) / (output_tp.max() - output_tp.min())
normalized_output = np.array(normalized_output)

input_list = get_input_data(normalized_input)
output_list = get_output_data(normalized_output)

# testデータの入力を用意
# normalized_test_input = (test_input_tp - test_input_tp.min()) / (test_input_tp.max() - test_input_tp.min())
# normalized_test_input = np.array(normalized_test_input)
# test_input_list = get_chunked_data(normalized_test_input, chunk_size)

# denormalize用
output_max = float(output_tp.max())
output_min = float(output_tp.min())

# RNNに突っ込むためにデータを整形
X = np.array(input_list).reshape(input_list.shape[0], input_list.shape[1])
Y = np.array(normalized_output).reshape(output_list.shape[0], output_list.shape[1])
# X_predict = np.array(test_input_list).reshape(len(test_input_list), chunk_size, test_input_list.shape[2])

In [85]:
Y.shape

(1460, 48)

In [86]:
X.shape

(1460, 246)

### **モデルの構築**

条件:翌日の気温を予測するには前日の20:00までの気象情報しか使えない

モデル:翌日の30分毎の気温を前日の20:00までの各所のアメダスの気象情報で予測させる

入力:各所のアメダス情報の配列

出力:0:00, 0:30, .... , 23:00, 23:30の気温の配列

とりあえず一つの箇所のデータを使って予測


In [110]:
# 今までLSTMやってたからとりあえずLSTMでやってみる

class MLP:
    def __init__(self, X, Y, epochs = 100, batch_size = 100, model_name = "test_model"):
        # 学習データと検証用データに分けておく
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=int((X.shape[0] * 0.1)))
        self.X = X # 入力
        self.Y = Y # 教師
        self.X_val = X_val # 検証用
        self.Y_val = Y_val #検証用
        
        '''
        諸変数の設定
        '''
        self.input_layer_size = self.X.shape[1] #入力層の数
        self.hidden_layer_size = 100 # 隠れ層の数、適当
        self.output_layer_size = self.Y.shape[1] #出力層の数
        self.batch_size = batch_size #バッチサイズ
        self.learning_rate = 0.01 # 学習率 適当
        self.epochs = epochs #エポック数
        
        # 学習データの保存
        self.model_name = str(model_name)
        
        
    def shuffle(self):
        '''
        ミニバッチかする際にランダムにシャッフル
        '''
        idx = [i for i in range(self.X.shape[0])]
        np.random.shuffle(idx)
        xs = np.array([[y for y in list(self.X[r])] for r in idx])
        ys = np.array([self.Y[r] for r in idx])
        return xs, ys
        
    def inference(self, input_ph):
        '''
        グラフの構築
        '''
        
        # 重みとバイアスの初期化
        hidden_w = tf.Variable(tf.truncated_normal([self.input_layer_size, self.hidden_layer_size], stddev=0.01), name='hidden_w')
        hidden_b = tf.Variable(tf.truncated_normal([self.hidden_layer_size]), name='hidden_b')
        output_w = tf.Variable(tf.truncated_normal([self.hidden_layer_size, self.output_layer_size], stddev=0.01), name='output_w')
        output_b = tf.Variable(tf.truncated_normal([self.output_layer_size]), name='output_b')
        
        # 計算
        print("input : ", input_ph)
        hidden = tf.sigmoid(tf.matmul(input_ph, hidden_w) + hidden_b)
        print("hidden : ", hidden)
        output = tf.sigmoid(tf.matmul(hidden, output_w) + output_b)
        print("output : ", output)
        
        weights = [hidden_w, output_w, hidden_w, hidden_b]
        
        return output, weights
        
    def loss(self, output_ph, actual_ph):
        '''
        MSEを使用
        '''
        cost = tf.reduce_mean(tf.square((output_ph - actual_ph)))
        tf.summary.scalar('loss', cost)
        return cost
    
    def training(self, cost):
        '''
        adamを仕様beta1, beta2は元論文の推奨値を仕様
        '''
        print("here")
        with tf.name_scope("training") as scope:
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate, beta1=0.9, beta2=0.999).minimize(cost)
            return optimizer
    
    def train(self):
        '''
        学習
        '''
        random.seed(0)
        np.random.seed(0)
        tf.set_random_seed(0)
        n_batch = self.X.shape[0] // self.batch_size
        
        with tf.Graph().as_default():
            # 変数の用意
            input_ph = tf.placeholder('float', [None, self.input_layer_size], name='input')
            actual_ph = tf.placeholder('float', [None, self.output_layer_size], name='actual_value')

            prediction, weights = self.inference(input_ph)
            cost = self.loss(prediction, actual_ph)
            optimizer = self.training(cost)

            # TensorBoardで可視化する
            summary = tf.summary.merge_all()
            # 初期化
            init = tf.global_variables_initializer()
                
            # ここから学習
            with tf.Session() as sess:
                # 学習したモデルも保存しておく
                saver = tf.train.Saver()
                summary_writer = tf.summary.FileWriter("/tmp/tensorflow_log", graph=sess.graph)
                sess.run(init)

                for epoch in range(self.epochs):
                    X_, Y_ = self.shuffle()
                    for i in range(n_batch):
                        start = i * self.batch_size
                        end = start + self.batch_size
                        inputs  = X_[start:end]
                        actuals = Y_[start:end]
                        train_dict = {
                            input_ph:      inputs,
                            actual_ph:     actuals,
                        }
                    
                    sess.run(optimizer, feed_dict=train_dict)

                    if (epoch) % (self.epochs//10) == 0:
                        val_dict = {
                            input_ph:      self.X_val,
                            actual_ph:     self.Y_val,
                        }
                        summary_str, train_loss = sess.run([summary, cost], feed_dict=val_dict)
                        print("train#%d, validation loss: %e" % (epoch, train_loss))
                        summary_writer.add_summary(summary_str, epoch)
                        
                    datas = sess.run(weights)
                    saver.save(sess,  "./data/" + str(self.model_name) + "/" + str(self.model_name) + ".ckpt")
                    
                datas = sess.run(weights)
                saver.save(sess, "./data/" + str(self.model_name) + "/" + str(self.model_name) + ".ckpt")
                
    def predict(self, X_predict, model_name = "test_model"):
        '''
        予測期間に該当するデータから予測
        '''
        # 予測に使う変数の用意
        tf.reset_default_graph()
        input_ph = tf.placeholder("float", [None, self.chunk_size, self.input_layer_size], name='input')
        prediction, weights = self.inference(input_ph)
        pre_dict = {
            input_ph: X_predict,
        }
        
        # 初期化
        init = tf.global_variables_initializer()

        with tf.Session() as sess:
            # 保存したモデルをロード
            saver = tf.train.Saver()
            saver.restore(sess,  "./data/" + str(self.model_name) + "/" + str(self.model_name) + ".ckpt")

            # ロードしたモデルを使って予測結果を計算
            expected_output = sess.run([prediction], feed_dict=pre_dict)


        return expected_output

In [111]:
mlp_model = MLP(X, Y, model_name = model_name)

In [112]:
mlp_model.train()

input :  Tensor("input:0", shape=(?, 246), dtype=float32)
hidden :  Tensor("Sigmoid:0", shape=(?, 100), dtype=float32)
output :  Tensor("Sigmoid_1:0", shape=(?, 48), dtype=float32)
here
train#0, validation loss: 2.099135e-01
train#10, validation loss: 1.075154e-01
train#20, validation loss: 1.065942e-01
train#30, validation loss: 1.000316e-01
train#40, validation loss: 9.780588e-02
train#50, validation loss: 1.013508e-01
train#60, validation loss: 9.752372e-02
train#70, validation loss: 9.135237e-02
train#80, validation loss: 8.894075e-02
train#90, validation loss: 8.764825e-02
